# Compilation du modèle avec ONNX

## 1 - Classes et fonctions

In [1]:
import pathlib

import torch
import torchvision
from collections import OrderedDict

In [2]:
class ReshapeToBatchChannelFirst(torch.nn.Module):
    def __init__(self):
        super(ReshapeToBatchChannelFirst, self).__init__()

    def forward(self, x):      
        # Permute dimensions from (H, W, C) to (C, H, W) and keep first 3 channels
        x = x.permute(2, 0, 1)[0:3]
        
        # Add a batch dimension at the beginning: (1, C, H, W)
        x = x.unsqueeze(0)
        return x

# Example usage
x = torch.rand(224, 224, 4)  # Example input
layer = ReshapeToBatchChannelFirst()
output = layer(x)
print(output.shape)  # Expected: torch.Size([1, 3, 224, 224])

torch.Size([1, 3, 224, 224])


In [3]:
class FixedNormLayer(torch.nn.Module):
    def __init__(self, scale=torch.tensor([1 / 256]), mean=torch.tensor([0.485, 0.456, 0.406]), std=torch.tensor([0.229, 0.224, 0.225])):
        """
        Args:
            mean (torch.Tensor): Precomputed mean for normalization.
            std (torch.Tensor): Precomputed standard deviation for normalization.
        """
        super(FixedNormLayer, self).__init__()
        self.register_buffer("mean", mean[:, None, None])
        self.register_buffer("std", std[:, None, None])
        self.register_buffer("scale", scale)

    def forward(self, x):
        return (self.scale * x - self.mean) / self.std

# Example usage
layer = FixedNormLayer()

# Test with a sample input
x = torch.rand(1, 3, 1, 1)  # Example input
output = layer(x)
print(output)

tensor([[[[-2.1115]],

         [[-2.0275]],

         [[-1.7896]]]])


In [4]:
class InferenceModelEncoder(torch.nn.Module):
    def __init__(self, model, scale=torch.tensor([1 / 256]), mean=torch.tensor([0.485, 0.456, 0.406]), std=torch.tensor([0.229, 0.224, 0.225])):
        super(InferenceModelEncoder, self).__init__()
        self.preprocess = torch.nn.Sequential(
            OrderedDict(
                [
                    ("reshape", ReshapeToBatchChannelFirst()),
                    ("normalize", FixedNormLayer(scale, mean, std)),
                ]
            )
        )
        self.model = model  # The main model

    def forward(self, x):
        x = self.preprocess(x)  # Apply reshaping and normalization
        x = self.model(x)  # Pass to the main model
        return x

In [5]:
class InferenceModelDecoder(torch.nn.Module):
    def __init__(self, model):
        super(InferenceModelDecoder, self).__init__()
        self.model = model  # The main model last layer
        self.postprocess = torch.nn.Softmax(1)

    def forward(self, x):
        x = self.model(x)  # Pass to the main model
        return self.postprocess(x)

In [6]:
import pathlib
from onnxruntime.tools import convert_onnx_models_to_ort as convert_onnx

def export_to_onnx(model, model_input, model_name, path=".", target_platform="arm"):
    path = pathlib.Path(path)

    # Export to ONNX format
    onnx_program = torch.onnx.export(model, model_input, dynamo=True)
    onnx_program.save((path / model_name).with_suffix(".onnx"))

    # Export to ORT format for mobile
    convert_onnx.convert_onnx_models_to_ort(
        (path / model_name).with_suffix(".onnx"),
        output_dir=path,
        optimization_styles=[convert_onnx.OptimizationStyle.Fixed],
        target_platform=target_platform,
    )

In [7]:
import PIL
import numpy as np

def single_image_pipeline(image_path, dtype="float32"):
    # Load image into numpy float array
    image = np.array(
        PIL.Image.open(image_path).convert("RGB").resize((224, 224)), dtype=dtype
    )

    return image

## 2 - Modèle généraliste (système encodeur-décodeur)

### 2.1 - Encodeur

In [12]:
# Load pretrained MobileNet_v3
num_labels = 18

model = torchvision.models.mobilenet_v3_large(weights="IMAGENET1K_V1")  # Load a pretrained model
model.classifier[3] = torch.nn.Linear(
    model.classifier[3].in_features, num_labels
)

# Load parameters
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.load_state_dict(
    torch.load(
        "models/generalist_model_2025_03_27.pt",
        map_location=device,
        weights_only=True,
    )
)

<All keys matched successfully>

In [13]:
# Copy classifier and set model classifier to identity
classifier = model.classifier

# Set model classifier to identity
model.classifier = torch.nn.Identity()

classifier

Sequential(
  (0): Linear(in_features=960, out_features=1280, bias=True)
  (1): Hardswish()
  (2): Dropout(p=0.2, inplace=True)
  (3): Linear(in_features=1280, out_features=18, bias=True)
)

In [14]:
# Define encoder and decoder inference models
inference_model_encoder = InferenceModelEncoder(model)
inference_model_decoder = InferenceModelDecoder(classifier)

inference_model_encoder.eval()
inference_model_decoder.eval()

InferenceModelDecoder(
  (model): Sequential(
    (0): Linear(in_features=960, out_features=1280, bias=True)
    (1): Hardswish()
    (2): Dropout(p=0.2, inplace=True)
    (3): Linear(in_features=1280, out_features=18, bias=True)
  )
  (postprocess): Softmax(dim=1)
)

In [15]:
# Test inference model encoder and decoder
torch_input = torch.randn(224, 224, 4)

encoded = inference_model_encoder(torch_input)
decoded = inference_model_decoder(encoded)
decoded

tensor([[0.0008, 0.0079, 0.0053, 0.0028, 0.0109, 0.0021, 0.0067, 0.0031, 0.0018,
         0.0063, 0.6355, 0.0387, 0.0087, 0.2508, 0.0058, 0.0103, 0.0010, 0.0015]],
       grad_fn=<SoftmaxBackward0>)

In [28]:
# Export to ONNX
path = pathlib.Path("onnx_models")
path.mkdir(exist_ok=True)

export_to_onnx(inference_model_encoder, torch_input, "generalist_model_2025_03_27_encoder", path)

[torch.onnx] Obtain model graph for `InferenceModelEncoder([...]` with `torch.export.export`...
[torch.onnx] Obtain model graph for `InferenceModelEncoder([...]` with `torch.export.export`... ✅
[torch.onnx] Translate the graph into ONNX...
[torch.onnx] Translate the graph into ONNX... ✅
Converting models with optimization style 'Fixed' and level 'all'
Converting optimized ONNX model /home/maxime/Documents/Code/happybud/training/onnx_models/generalist_model_2025_03_27_encoder.onnx to ORT format model /home/maxime/Documents/Code/happybud/training/onnx_models/generalist_model_2025_03_27_encoder.ort


2025-04-17 12:32:43.274732504 [W:onnxruntime:, graph.cc:109 MergeShapeInfo] Error merging shape info for output. '_native_batch_norm_legit_no_training__1' source:{16} target:{0}. Falling back to lenient merge.
2025-04-17 12:32:43.274912095 [W:onnxruntime:, graph.cc:109 MergeShapeInfo] Error merging shape info for output. '_native_batch_norm_legit_no_training__2' source:{16} target:{0}. Falling back to lenient merge.
2025-04-17 12:32:43.276012540 [W:onnxruntime:, graph.cc:109 MergeShapeInfo] Error merging shape info for output. '_native_batch_norm_legit_no_training_1__1' source:{16} target:{0}. Falling back to lenient merge.
2025-04-17 12:32:43.276130035 [W:onnxruntime:, graph.cc:109 MergeShapeInfo] Error merging shape info for output. '_native_batch_norm_legit_no_training_1__2' source:{16} target:{0}. Falling back to lenient merge.
2025-04-17 12:32:43.277082648 [W:onnxruntime:, graph.cc:109 MergeShapeInfo] Error merging shape info for output. '_native_batch_norm_legit_no_training_2__1'

Converted 1/1 models successfully.
Generating config file from ORT format models with optimization style 'Fixed' and level 'all'


### 2.2 - Décodeur des plantes

In [29]:
# Load pretrained MobileNet_v3
num_labels = 3

model = torchvision.models.mobilenet_v3_large(weights="IMAGENET1K_V1")  # Load a pretrained model
model.classifier[3] = torch.nn.Linear(
    model.classifier[3].in_features, num_labels
)

# Load parameters
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.load_state_dict(
    torch.load(
        "models/generalist_model_plants_2025_03_27.pt",
        map_location=device,
        weights_only=True,
    )
)

<All keys matched successfully>

In [30]:
# Copy classifier and set model classifier to identity
classifier = model.classifier

classifier

Sequential(
  (0): Linear(in_features=960, out_features=1280, bias=True)
  (1): Hardswish()
  (2): Dropout(p=0.2, inplace=True)
  (3): Linear(in_features=1280, out_features=3, bias=True)
)

In [31]:
# Define decoder inference models
inference_model_decoder = InferenceModelDecoder(classifier)
inference_model_decoder.eval()

InferenceModelDecoder(
  (model): Sequential(
    (0): Linear(in_features=960, out_features=1280, bias=True)
    (1): Hardswish()
    (2): Dropout(p=0.2, inplace=True)
    (3): Linear(in_features=1280, out_features=3, bias=True)
  )
  (postprocess): Softmax(dim=1)
)

In [32]:
# Export to ONNX
export_to_onnx(inference_model_decoder, encoded, "generalist_model_plants_2025_03_27_decoder", path)

[torch.onnx] Obtain model graph for `InferenceModelDecoder([...]` with `torch.export.export`...
[torch.onnx] Obtain model graph for `InferenceModelDecoder([...]` with `torch.export.export`... ✅
[torch.onnx] Translate the graph into ONNX...


2025-04-17 12:34:27,925 ort_format_model.utils [INFO] - Created config in /home/maxime/Documents/Code/happybud/training/onnx_models/generalist_model_plants_2025_03_27_decoder.required_operators.config


[torch.onnx] Translate the graph into ONNX... ✅
Converting models with optimization style 'Fixed' and level 'all'
Converting optimized ONNX model /home/maxime/Documents/Code/happybud/training/onnx_models/generalist_model_plants_2025_03_27_decoder.onnx to ORT format model /home/maxime/Documents/Code/happybud/training/onnx_models/generalist_model_plants_2025_03_27_decoder.ort
Converted 1/1 models successfully.
Generating config file from ORT format models with optimization style 'Fixed' and level 'all'


### 2.3 - Décodeur des tomates

In [26]:
# Load pretrained MobileNet_v3
num_labels = 10

model = torchvision.models.mobilenet_v3_large(weights="IMAGENET1K_V1")  # Load a pretrained model
model.classifier[3] = torch.nn.Linear(
    model.classifier[3].in_features, num_labels
)

# Load parameters
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.load_state_dict(
    torch.load(
        "models/generalist_model_tomato_2025_03_27.pt",
        map_location=device,
        weights_only=True,
    )
)

<All keys matched successfully>

In [27]:
# Copy classifier and set model classifier to identity
classifier = model.classifier

classifier

Sequential(
  (0): Linear(in_features=960, out_features=1280, bias=True)
  (1): Hardswish()
  (2): Dropout(p=0.2, inplace=True)
  (3): Linear(in_features=1280, out_features=10, bias=True)
)

In [28]:
# Define decoder inference models
inference_model_decoder = InferenceModelDecoder(classifier)
inference_model_decoder.eval()

InferenceModelDecoder(
  (model): Sequential(
    (0): Linear(in_features=960, out_features=1280, bias=True)
    (1): Hardswish()
    (2): Dropout(p=0.2, inplace=True)
    (3): Linear(in_features=1280, out_features=10, bias=True)
  )
  (postprocess): Softmax(dim=1)
)

In [29]:
# Export to ONNX
export_to_onnx(inference_model_decoder, encoded, "generalist_model_tomato_2025_03_27_decoder", path)

[torch.onnx] Obtain model graph for `InferenceModelDecoder([...]` with `torch.export.export`...
[torch.onnx] Obtain model graph for `InferenceModelDecoder([...]` with `torch.export.export`... ✅
[torch.onnx] Translate the graph into ONNX...


2025-04-24 10:55:49,616 ort_format_model.utils [INFO] - Created config in /home/maxime/Documents/Code/happybud/training/onnx_models/generalist_model_tomato_2025_03_27_decoder.required_operators.config


[torch.onnx] Translate the graph into ONNX... ✅
Converting models with optimization style 'Fixed' and level 'all'
Converting optimized ONNX model /home/maxime/Documents/Code/happybud/training/onnx_models/generalist_model_tomato_2025_03_27_decoder.onnx to ORT format model /home/maxime/Documents/Code/happybud/training/onnx_models/generalist_model_tomato_2025_03_27_decoder.ort
Converted 1/1 models successfully.
Generating config file from ORT format models with optimization style 'Fixed' and level 'all'


### 2.4 - Décodeur du maïs

In [16]:
# Load pretrained MobileNet_v3
num_labels = 5

model = torchvision.models.mobilenet_v3_large(weights="IMAGENET1K_V1")  # Load a pretrained model
model.classifier[3] = torch.nn.Linear(
    model.classifier[3].in_features, num_labels
)

# Load parameters
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.load_state_dict(
    torch.load(
        "models/generalist_model_corn_2025_04_22.pt",
        map_location=device,
        weights_only=True,
    )
)

<All keys matched successfully>

In [17]:
# Copy classifier and set model classifier to identity
classifier = model.classifier

classifier

Sequential(
  (0): Linear(in_features=960, out_features=1280, bias=True)
  (1): Hardswish()
  (2): Dropout(p=0.2, inplace=True)
  (3): Linear(in_features=1280, out_features=5, bias=True)
)

In [18]:
# Define decoder inference models
inference_model_decoder = InferenceModelDecoder(classifier)
inference_model_decoder.eval()

InferenceModelDecoder(
  (model): Sequential(
    (0): Linear(in_features=960, out_features=1280, bias=True)
    (1): Hardswish()
    (2): Dropout(p=0.2, inplace=True)
    (3): Linear(in_features=1280, out_features=5, bias=True)
  )
  (postprocess): Softmax(dim=1)
)

In [20]:
# Export to ONNX
path = pathlib.Path("onnx_models")
export_to_onnx(inference_model_decoder, encoded, "generalist_model_corn_2025_04_22_decoder", path)

/home/maxime/.venvs/ai/lib64/python3.11/site-packages/onnxscript/converter.py:823: FutureWarning: 'onnxscript.values.Op.param_schemas' is deprecated in version 0.1 and will be removed in the future. Please use '.op_signature' instead.
  param_schemas = callee.param_schemas()
/home/maxime/.venvs/ai/lib64/python3.11/site-packages/onnxscript/converter.py:823: FutureWarning: 'onnxscript.values.OnnxFunction.param_schemas' is deprecated in version 0.1 and will be removed in the future. Please use '.op_signature' instead.
  param_schemas = callee.param_schemas()


[torch.onnx] Obtain model graph for `InferenceModelDecoder([...]` with `torch.export.export`...
[torch.onnx] Obtain model graph for `InferenceModelDecoder([...]` with `torch.export.export`... ✅
[torch.onnx] Translate the graph into ONNX...


2025-04-24 10:54:41,005 ort_format_model.utils [INFO] - Created config in /home/maxime/Documents/Code/happybud/training/onnx_models/generalist_model_corn_2025_04_22_decoder.required_operators.config


[torch.onnx] Translate the graph into ONNX... ✅
Converting models with optimization style 'Fixed' and level 'all'
Converting optimized ONNX model /home/maxime/Documents/Code/happybud/training/onnx_models/generalist_model_corn_2025_04_22_decoder.onnx to ORT format model /home/maxime/Documents/Code/happybud/training/onnx_models/generalist_model_corn_2025_04_22_decoder.ort
Converted 1/1 models successfully.
Generating config file from ORT format models with optimization style 'Fixed' and level 'all'


### 2.5 - Décodeur des patates

In [21]:
# Load pretrained MobileNet_v3
num_labels = 3

model = torchvision.models.mobilenet_v3_large(weights="IMAGENET1K_V1")  # Load a pretrained model
model.classifier[3] = torch.nn.Linear(
    model.classifier[3].in_features, num_labels
)

# Load parameters
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.load_state_dict(
    torch.load(
        "models/generalist_model_potato_2025_04_22.pt",
        map_location=device,
        weights_only=True,
    )
)

<All keys matched successfully>

In [22]:
# Copy classifier and set model classifier to identity
classifier = model.classifier

classifier

Sequential(
  (0): Linear(in_features=960, out_features=1280, bias=True)
  (1): Hardswish()
  (2): Dropout(p=0.2, inplace=True)
  (3): Linear(in_features=1280, out_features=3, bias=True)
)

In [23]:
# Define decoder inference models
inference_model_decoder = InferenceModelDecoder(classifier)
inference_model_decoder.eval()

InferenceModelDecoder(
  (model): Sequential(
    (0): Linear(in_features=960, out_features=1280, bias=True)
    (1): Hardswish()
    (2): Dropout(p=0.2, inplace=True)
    (3): Linear(in_features=1280, out_features=3, bias=True)
  )
  (postprocess): Softmax(dim=1)
)

In [25]:
# Export to ONNX
export_to_onnx(inference_model_decoder, encoded, "generalist_model_potato_2025_04_22_decoder", path)

[torch.onnx] Obtain model graph for `InferenceModelDecoder([...]` with `torch.export.export`...
[torch.onnx] Obtain model graph for `InferenceModelDecoder([...]` with `torch.export.export`... ✅
[torch.onnx] Translate the graph into ONNX...


2025-04-24 10:55:26,790 ort_format_model.utils [INFO] - Created config in /home/maxime/Documents/Code/happybud/training/onnx_models/generalist_model_potato_2025_04_22_decoder.required_operators.config


[torch.onnx] Translate the graph into ONNX... ✅
Converting models with optimization style 'Fixed' and level 'all'
Converting optimized ONNX model /home/maxime/Documents/Code/happybud/training/onnx_models/generalist_model_potato_2025_04_22_decoder.onnx to ORT format model /home/maxime/Documents/Code/happybud/training/onnx_models/generalist_model_potato_2025_04_22_decoder.ort
Converted 1/1 models successfully.
Generating config file from ORT format models with optimization style 'Fixed' and level 'all'


## 3 - Nettoyage

In [30]:
for file in path.glob("*.config"):
    file.unlink()